In [0]:
import pandas as pd 
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd 
import numpy as np

df_additional_train = pd.read_csv('/content/drive/My Drive/DS_UMA/data_additional_prepared_train.csv') # чтение из ранее подготовленных 
df_tr = pd.read_csv('/content/drive/My Drive/DS_UMA/TRAIN_PREPARED.csv') # источников
df_train = df_tr.reset_index().merge(df_additional_train, how='inner', on='user', sort=False) #мердж по пользователю
df_additional_test = pd.read_csv('/content/drive/My Drive/DS_UMA/data_additional_prepared_test.csv') 
df_tst = pd.read_csv('/content/drive/My Drive/DS_UMA/TEST_PREPARED.csv')
df_test = df_tst.reset_index().merge(df_additional_test, how='inner', on='user', sort=False)
delim = df_train.shape[0] # для последующего разделения объединенных выборок

In [228]:
df_common = pd.concat([df_train,df_test])

0    e98a93bfc7
Name: user, dtype: object

In [0]:
df_common['Difference'] = pd.to_datetime(df_common['last_date']) -  pd.to_datetime(df_common['first_date'])
df_common.Difference = df_common.Difference.dt.days
df_common['hash_user'] = df_common['user'].apply(hash) # для использования в качестве числового значения 
df_common = df_common.set_index('cut_date')
test_user = df_common[delim:].user
df_common = df_common.drop(columns=['user','first_date','last_date','calday'])

In [0]:
Y_train = df_common[:delim].label
X_train = df_common[:delim].drop(columns = 'label')
X_test = df_common[delim:].drop(columns = 'label')

In [0]:
clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',  # обучение с выбранными ранее на GridSearchCV параметрами
                           n_neighbors=3, p=2,
                     weights='distance') # модель на кросс валидации показала лучшее качество 
clf.fit(X_train,Y_train)
predictions = clf.predict(X_test)

In [0]:
df_test = pd.DataFrame(test_user)
df_test['predictions'] = predictions

Большая просьба оставить фидбэк, как стоит доработать модель, какие есть ошибки и т.д. Заранее спасибо!

In [0]:
df_test = df_test.drop_duplicates()
df_test.to_csv('predictions.csv')